In [1]:
name = "2020-09-10-github-scrape"
title = "Scraping GitHub after a hackweek"
tags = "requests, github, webscrape, pandas"
author = "Callum Rollo"

In [2]:
from nb_tools import connect_notebook_to_post
from IPython.core.display import HTML

html = connect_notebook_to_post(name, title, tags, author)

### A meta-hackweek hack

I put this notebook together after attending the excellent [Oceean Hack Week 2020](https://oceanhackweek.github.io/) (OHW) event. You can read my blog post about it [here](https://callumrollo.github.io/hackweek.html#hackweek). A key part of the event was creating collaborative projects on GitHub. I wanted to see if attending the hackweek changed participants pattern of activity on GitHub. I though the simplest way would be to get info on all the commits by OHW participants to OHW projects.

It goes without saying that **commits =/= work done on a project**. I freely admit that many of my commits are nonsense! This is just a fun side project for me to explore the github API and try some simple analysis methods.

First off, to access the [Github API](https://docs.github.com/) you'll need to edit the credentials file `credentials.json` to supply your username and a [Github access token](https://github.blog/2013-05-16-personal-api-tokens/).

```
{
	"username": "<your-username>",
	"token": "<your-access-token>"
}
```

Once you have supplied these creds, you are still limited to [5000 requests per hour](https://docs.github.com/en/rest/overview/resources-in-the-rest-api#rate-limiting) so, if you get bounced by the API, leave it some time to cool off. Without credentials the limit is far lower and you will would soon generate an error message like this:

![rate limit reached](../figures/rate-limit-anon.png)

Obviously I have not provided my own credentials! I'm not sure what action github would take if a DDOS on their API originated from my account, but I'm not willing to find out.

### Let's get scraping!

In [1]:
import json
import requests
from collections import Counter
import pandas as pd
import numpy as np

In [2]:
credentials = json.loads(open('credentials-secret.json').read()) #don't forget to add your creds here!

username = credentials['username']
token = credentials['token']

For a start, let's use the API to get some details on my account

In [3]:
user_data = requests.get('https://api.github.com/users/' + credentials['username'],auth = (username,token)).json()
user_data

{'login': 'callumrollo',
 'id': 28703282,
 'node_id': 'MDQ6VXNlcjI4NzAzMjgy',
 'avatar_url': 'https://avatars0.githubusercontent.com/u/28703282?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/callumrollo',
 'html_url': 'https://github.com/callumrollo',
 'followers_url': 'https://api.github.com/users/callumrollo/followers',
 'following_url': 'https://api.github.com/users/callumrollo/following{/other_user}',
 'gists_url': 'https://api.github.com/users/callumrollo/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/callumrollo/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/callumrollo/subscriptions',
 'organizations_url': 'https://api.github.com/users/callumrollo/orgs',
 'repos_url': 'https://api.github.com/users/callumrollo/repos',
 'events_url': 'https://api.github.com/users/callumrollo/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/callumrollo/received_events',
 'type': 'User',
 'site_admin': False,
 'n

This returns a lot of info. Now, I'll try the account of one of my collaborators `ocefpaf`. You can specify any user, though the information returned is less than when you look at your own account.

In [4]:
data = requests.get('https://api.github.com/users/' + 'ocefpaf',auth = (username,token)).json()
data

{'login': 'ocefpaf',
 'id': 950575,
 'node_id': 'MDQ6VXNlcjk1MDU3NQ==',
 'avatar_url': 'https://avatars1.githubusercontent.com/u/950575?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/ocefpaf',
 'html_url': 'https://github.com/ocefpaf',
 'followers_url': 'https://api.github.com/users/ocefpaf/followers',
 'following_url': 'https://api.github.com/users/ocefpaf/following{/other_user}',
 'gists_url': 'https://api.github.com/users/ocefpaf/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/ocefpaf/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/ocefpaf/subscriptions',
 'organizations_url': 'https://api.github.com/users/ocefpaf/orgs',
 'repos_url': 'https://api.github.com/users/ocefpaf/repos',
 'events_url': 'https://api.github.com/users/ocefpaf/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/ocefpaf/received_events',
 'type': 'User',
 'site_admin': False,
 'name': 'Filipe',
 'company': None,
 'blog': 'http://o

We can see a user's core stats. How about their commits and other actions taken? Simply append `/events` to the request query

### Events

In [5]:
data = requests.get('https://api.github.com/users/' + 'callumrollo' +'/events',auth = (username,token)).json()
data[0]

{'id': '13616867582',
 'type': 'PushEvent',
 'actor': {'id': 28703282,
  'login': 'callumrollo',
  'display_login': 'callumrollo',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/callumrollo',
  'avatar_url': 'https://avatars.githubusercontent.com/u/28703282?'},
 'repo': {'id': 298024499,
  'name': 'callumrollo/adcp-glider-guide',
  'url': 'https://api.github.com/repos/callumrollo/adcp-glider-guide'},
 'payload': {'push_id': 5738901395,
  'size': 1,
  'distinct_size': 1,
  'ref': 'refs/heads/main',
  'head': '06de9d302a0de253babf425ca48c0dc1d4648e35',
  'before': '07d98d0de85a094da6455574371c700bf8df28db',
  'commits': [{'sha': '06de9d302a0de253babf425ca48c0dc1d4648e35',
    'author': {'email': 'c.rollo@outlook.com', 'name': 'Callum Rollo'},
    'message': 'point to pdf for reading',
    'distinct': True,
    'url': 'https://api.github.com/repos/callumrollo/adcp-glider-guide/commits/06de9d302a0de253babf425ca48c0dc1d4648e35'}]},
 'public': True,
 'created_at': '2020-09-23T16

Note that the Github API lists only the 30 most recent events

In [6]:
len(data)

30

To get at more events, we use a short loop to access subsequent pages of results. I found out the hard way that the API restricts you to 10 pages.

In [7]:
tgt_user = 'callumrollo'
base_url = 'https://api.github.com/users/' + tgt_user +'/events'
url = base_url
url_list = [base_url]
data = []
page_no = 1
repos_data = []
total_fetched = 0
while (True):
    response = requests.get(url,auth = (username,token)).json()
    data = data + response
    events_fetched = len(response)
    total_fetched = events_fetched + total_fetched
    print(f"Page: {page_no} total events fetched: {total_fetched}")
    
    if total_fetched == 300:
        print(f"\nAPI maxed out! https://docs.github.com/v3/#pagination\n\
        returning only most recent 300 events by {tgt_user}")
        print(f"\nevents span the range \n{data[-1]['created_at']}\n{data[0]['created_at']}")
        break
    
    if (events_fetched == 30):
        page_no = page_no + 1
        url = base_url + '?page=' + str(page_no)
        url_list.append(url)
    else:
        print(f"\n{tgt_user}: all your events are belong to us now")
        print(f"\nevents span the range \n{data[-1]['created_at']}\n{data[0]['created_at']}")
        break

Page: 1 total events fetched: 30
Page: 2 total events fetched: 60
Page: 3 total events fetched: 90
Page: 4 total events fetched: 120
Page: 5 total events fetched: 150
Page: 6 total events fetched: 180
Page: 7 total events fetched: 210
Page: 8 total events fetched: 240
Page: 9 total events fetched: 259

callumrollo: all your events are belong to us now

events span the range 
2020-06-23T16:12:03Z
2020-09-23T16:11:06Z


This system logs all events: commits, issues, PRs, forks, stars etc. We are only interested in commits.

These are referred to as `PushEvent` in the json entry `type`

In [8]:
for event in data:
    print(event["type"])


PushEvent
CreateEvent
CreateEvent
WatchEvent
PushEvent
PushEvent
PushEvent
CreateEvent
ForkEvent
WatchEvent
WatchEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
CreateEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
CreateEvent
CreateEvent
WatchEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
WatchEvent
CreateEvent
CreateEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
IssuesEvent
WatchEvent
WatchEvent
PushEvent
PushEvent
PushEvent
PullRequestEvent
PushEvent
PullRequestEvent
PushEvent
PullRequestEvent
PushEvent
PushEvent
PullRequestEvent
PullRequestEvent
PushEvent
PullRequestEvent
CreateEvent
PushEvent
PullRequestEvent
CreateEvent
PushEvent
IssuesEvent
IssuesEvent
PushEvent
PushEvent
PullRequestEvent
PullRequestEvent
CreateEvent
PushEvent
CreateEvent
PushEvent
IssueCommentEvent
IssueCommentEvent
Issu

In [9]:
commit_events = []
for event in data:
    if event["type"] == "PushEvent":
        commit_events.append(event)
len(commit_events)

177

There are some complications. Not all of these commits in these events are by the Github user we are querying. For instance, some are commits by other users that our target user has merged in.

To work around this, we look through the payoad of each `PushEvent` and retain only the commits associated with the user we are interested in.

**n.b** this approach wil only work if the github username is the exact match for the name the author uses for their git commits. We apply a workaround for when this is not the case later

In [10]:
tgt_username = requests.get('https://api.github.com/users/' + credentials['username'],
                            auth = (username,token)).json()["name"]
tgt_username

'Callum Rollo'

In [11]:
user_commits = []
for event in commit_events:
    commit_list = event["payload"]["commits"]
    commit_list_author = []
    if len(commit_list)>0:
        for com_n in range(len(commit_list)):
            commit_username = commit_list[com_n]["author"]["name"]
            #print(commit_username)
            if commit_username == tgt_username:
                commit_list_author.append(commit_list[com_n])
        if commit_list_author:
            event["payload"]["commits"] = commit_list_author
            user_commits.append(event)

In [12]:
len(user_commits)

174

We can print out the name associated with the commits we have selected to confirm

In [13]:
commit_names = []
for event in user_commits:
    commit_list = event["payload"]["commits"]
    for com_n in range(len(commit_list)):
            commit_username = commit_list[com_n]["author"]["name"]
            commit_names.append(commit_username)
print("git usernames and number of commits:")
Counter(commit_names).most_common()

git usernames and number of commits:


[('Callum Rollo', 225)]

In [14]:
print(f"From {len(data)} events we have extracted {len(commit_names)} commits by {tgt_username}")

From 259 events we have extracted 225 commits by Callum Rollo


The final step of this (almost certainly imperfect) data cleaning is to get info on all the commits by this user. We will pull the author, message, SHA, url, repo and date into a pandas dataframe. 

In [15]:
df = pd.DataFrame()
for event in user_commits:
    for commit in event["payload"]["commits"]:
        commit_subset = {"id": event["id"],
                     "datetime" : event["created_at"],
                     "sha" : commit["sha"],
                     "message" : commit["message"],
                     "author" : commit["author"]["name"],
                     "url": commit["url"],
                     "repo": event["repo"]["name"]}
        df = df.append(commit_subset, ignore_index=True)

We index by datetime and have a look at our dataframe

In [16]:
df.index = pd.DatetimeIndex(df.datetime)
df = df.drop("datetime", axis=1)
df

,author,id,message,repo,sha,url
datetime,,,,,,
2020-09-05 19:16:44+00:00,Callum Rollo,13422927661,Finalised scraper function,callumrollo/github-scraper,78d24e1e104f616f8026a73a10fd525e67019342,https://api.github.com/repos/callumrollo/githu...
2020-09-05 19:16:44+00:00,Callum Rollo,13422927661,detailed time breakdown histograms,callumrollo/github-scraper,2445a3d0d800bc3fea933d713ae5cca173bd0f72,https://api.github.com/repos/callumrollo/githu...
2020-09-05 17:21:13+00:00,Callum Rollo,13422558148,Generate Pelican site,callumrollo/callumrollo.github.io,a9e3bb9213139f467fb5058847ff063e851b1ce6,https://api.github.com/repos/callumrollo/callu...
2020-09-05 17:20:55+00:00,Callum Rollo,13422557015,updated categories,callumrollo/callumrollo.github.io,aca1fa72ad196df5351661a971b7de8add348fc6,https://api.github.com/repos/callumrollo/callu...
2020-09-05 16:52:01+00:00,Callum Rollo,13422458592,Generate Pelican site,callumrollo/callumrollo.github.io,79a6e0329968330e91bd9d202fac5257ce5c9ab2,https://api.github.com/repos/callumrollo/callu...
...,...,...,...,...,...,...
2020-06-08 12:32:28+00:00,Callum Rollo,12567167762,ENV East >> Aries,ueapy/pythoncourse2020,63f61e7f654217d5cdc1af9925de2ed593fe154e,https://api.github.com/repos/ueapy/pythoncours...
2020-06-08 10:52:40+00:00,Callum Rollo,12566251693,typo,callumrollo/callumrollo.github.io,e712699f1d08f2294bd7e462bcb92768f6841e04,https://api.github.com/repos/callumrollo/callu...
2020-06-08 10:49:46+00:00,Callum Rollo,12566225573,nice readme,callumrollo/callumrollo.github.io,7f4e591fd2dd9fe043f240430ba3bf7bb0a4d062,https://api.github.com/repos/callumrollo/callu...


We can see an uncharacteristically helpful set of commit messages, and the meta event of a commit I made to this very notebook

Now we remove any repeated commits that may have snuck in by a deduplicating on the SHA checksum

**side note** the SHA chescksum uniquely identifies each commit. Even if you had commits by the same author to the same repo with the same message ("added stuff" or something similarly helpful) the SHA will differentiate the two. See more [here](https://www.lifewire.com/what-is-sha-1-2626011)

In [17]:
df = df.drop_duplicates(subset=['sha'])
df

,author,id,message,repo,sha,url
datetime,,,,,,
2020-09-05 19:16:44+00:00,Callum Rollo,13422927661,Finalised scraper function,callumrollo/github-scraper,78d24e1e104f616f8026a73a10fd525e67019342,https://api.github.com/repos/callumrollo/githu...
2020-09-05 19:16:44+00:00,Callum Rollo,13422927661,detailed time breakdown histograms,callumrollo/github-scraper,2445a3d0d800bc3fea933d713ae5cca173bd0f72,https://api.github.com/repos/callumrollo/githu...
2020-09-05 17:21:13+00:00,Callum Rollo,13422558148,Generate Pelican site,callumrollo/callumrollo.github.io,a9e3bb9213139f467fb5058847ff063e851b1ce6,https://api.github.com/repos/callumrollo/callu...
2020-09-05 17:20:55+00:00,Callum Rollo,13422557015,updated categories,callumrollo/callumrollo.github.io,aca1fa72ad196df5351661a971b7de8add348fc6,https://api.github.com/repos/callumrollo/callu...
2020-09-05 16:52:01+00:00,Callum Rollo,13422458592,Generate Pelican site,callumrollo/callumrollo.github.io,79a6e0329968330e91bd9d202fac5257ce5c9ab2,https://api.github.com/repos/callumrollo/callu...
...,...,...,...,...,...,...
2020-06-08 12:32:28+00:00,Callum Rollo,12567167762,ENV East >> Aries,ueapy/pythoncourse2020,63f61e7f654217d5cdc1af9925de2ed593fe154e,https://api.github.com/repos/ueapy/pythoncours...
2020-06-08 10:52:40+00:00,Callum Rollo,12566251693,typo,callumrollo/callumrollo.github.io,e712699f1d08f2294bd7e462bcb92768f6841e04,https://api.github.com/repos/callumrollo/callu...
2020-06-08 10:49:46+00:00,Callum Rollo,12566225573,nice readme,callumrollo/callumrollo.github.io,7f4e591fd2dd9fe043f240430ba3bf7bb0a4d062,https://api.github.com/repos/callumrollo/callu...


------------------------
### Scaling it up: work from a list of Github usernames

Now that we have a method for finding commits by a user, the next step is to loop through a list of users. As a test case, I have analysed the commits from [Oceanhackweek2020](https://oceanhackweek.github.io/)

In [18]:
def gh_scrape(tgt_users, cred_file = 'credentials.json', verbose=True):
    """Simple scraping function
    Supply a list of github usersnames ['jane-doe', 'torvalds', 'satoshi_nakamoto']
    Returns a dataframe of unique commits by these users over the last 90 days
    Assumes that the github user is the user with most git commits associated with their github profile
    Rate limited by the Github API to 300 events
    Requires you to supply a Github API token in a credentials.json file
    Verbose switch prints a line for each user with the number of events and commits found
    Returns a pandas dataframe of commit info for all usernames in supplied list
    """
    # Get user supplied credentials for the github API
    credentials = json.loads(open(cred_file).read())
    username = credentials['username']
    token = credentials['token']
    df = pd.DataFrame()
    
    for tgt_user in tgt_users:
        base_url = 'https://api.github.com/users/' + tgt_user +'/events'
        url = base_url
        url_list = [base_url]
        data = []
        page_no = 1
        repos_data = []
        total_fetched = 0
        while (True):
            response = requests.get(url,auth = (username,token)).json()
            if type(response)==dict:
                # Catch when the API returns a dict rather than expected list. Usually a credentials error message
                print(response)
                return
            data = data + response
            events_fetched = len(response)
            total_fetched = events_fetched + total_fetched
            if total_fetched == 300:
                # Requesting more will max out the API
                break
            if (events_fetched == 30):
                # if we fethced 30 events from this page, there will be another one after it
                page_no = page_no + 1
                url = base_url + '?page=' + str(page_no)
                url_list.append(url)
            else:
                # We have collected all events by this user
                break
        commits_events = []
        for event in data:
            # We're only interested in commits, which are classed as "PushEvents"
            if event["type"] == "PushEvent":
                commits_events.append(event)
        if len(commits_events)==0:
            # If the user has no commit events, stop processing
            continue
            
        commit_usernames_list = []
        for event in commits_events:
            # Search though the payload for which git user is associated with each commit
            commit_list = event["payload"]["commits"]
            if len(commit_list)>0:    
                for com_n in range(len(commit_list)):
                        commit_username = commit_list[com_n]["author"]["name"]       
                        commit_usernames_list.append(commit_username)
        # Working on the assumption that the git user with the most commits pushed to Github by this user is the one we want
        c = Counter(commit_usernames_list)
        most_common_username = c.most_common(1)[0][0]
        user_commits = []
        
        # Go back through the commits and pull only the ones by the most common git username
        for event in commits_events:
            commit_list = event["payload"]["commits"]
            commit_list_author = []
            if len(commit_list)>0:    
                for com_n in range(len(commit_list)):
                    commit_username = commit_list[com_n]["author"]["name"]
                    if commit_username == most_common_username:
                        commit_list_author.append(commit_list[com_n])
                if commit_list_author:
                    event["payload"]["commits"] = commit_list_author
                    user_commits.append(event)
        
        # Extract the information we're interested in and put it in a pandas DataFrame
        for commit in user_commits:
            for com_n in range(len(commit["payload"]["commits"])):
                commit_detail = commit["payload"]["commits"][com_n]
                commit_subset = {"id": commit["id"],
                             "datetime" : commit["created_at"],
                             "sha" : commit_detail["sha"],
                             "message" : commit_detail["message"],
                             "author" : commit_detail["author"]["name"],
                             "url": commit_detail["url"],
                             "repo": commit["repo"]["name"]}
                df = df.append(commit_subset, ignore_index=True)
        df.index = pd.DatetimeIndex(df.datetime)
        df = df.drop_duplicates(subset=['sha'])
        if verbose:
            print(f"{tgt_user}: found {len(data)} events containing {len(user_commits)} unique commits by {most_common_username}\n")
    return df


In [19]:
users = ["callumrollo"]
df = gh_scrape(users, cred_file = 'credentials-secret.json')
df

callumrollo: found 300 events containing 214 unique commits by Callum Rollo



,author,datetime,id,message,repo,sha,url
datetime,,,,,,,
2020-09-05 19:16:44+00:00,Callum Rollo,2020-09-05T19:16:44Z,13422927661,Finalised scraper function,callumrollo/github-scraper,78d24e1e104f616f8026a73a10fd525e67019342,https://api.github.com/repos/callumrollo/githu...
2020-09-05 19:16:44+00:00,Callum Rollo,2020-09-05T19:16:44Z,13422927661,detailed time breakdown histograms,callumrollo/github-scraper,2445a3d0d800bc3fea933d713ae5cca173bd0f72,https://api.github.com/repos/callumrollo/githu...
2020-09-05 17:21:13+00:00,Callum Rollo,2020-09-05T17:21:13Z,13422558148,Generate Pelican site,callumrollo/callumrollo.github.io,a9e3bb9213139f467fb5058847ff063e851b1ce6,https://api.github.com/repos/callumrollo/callu...
2020-09-05 17:20:55+00:00,Callum Rollo,2020-09-05T17:20:55Z,13422557015,updated categories,callumrollo/callumrollo.github.io,aca1fa72ad196df5351661a971b7de8add348fc6,https://api.github.com/repos/callumrollo/callu...
2020-09-05 16:52:01+00:00,Callum Rollo,2020-09-05T16:52:01Z,13422458592,Generate Pelican site,callumrollo/callumrollo.github.io,79a6e0329968330e91bd9d202fac5257ce5c9ab2,https://api.github.com/repos/callumrollo/callu...
...,...,...,...,...,...,...,...
2020-06-08 12:32:28+00:00,Callum Rollo,2020-06-08T12:32:28Z,12567167762,ENV East >> Aries,ueapy/pythoncourse2020,63f61e7f654217d5cdc1af9925de2ed593fe154e,https://api.github.com/repos/ueapy/pythoncours...
2020-06-08 10:52:40+00:00,Callum Rollo,2020-06-08T10:52:40Z,12566251693,typo,callumrollo/callumrollo.github.io,e712699f1d08f2294bd7e462bcb92768f6841e04,https://api.github.com/repos/callumrollo/callu...
2020-06-08 10:49:46+00:00,Callum Rollo,2020-06-08T10:49:46Z,12566225573,nice readme,callumrollo/callumrollo.github.io,7f4e591fd2dd9fe043f240430ba3bf7bb0a4d062,https://api.github.com/repos/callumrollo/callu...


Quite a few commits. What if we want solely the hackweek ones?

In [20]:
df['ohw20_repo'] = df['repo'].str.contains("ohw20")
sum(df['ohw20_repo'] )

19

### OHW analysis

Using the above function and a list of hackweek participants (not included) I grab github commits from the last 90 days

In [16]:
import csv
from itertools import chain
with open('ohw_participants.csv', newline='') as f:
    nest_list = list(csv.reader(f))
ohw_participants_list = list(chain.from_iterable(nest_list))
df_all = gh_scrape(ohw_participants_list, cred_file='credentials-secret.json', verbose=False)

KeyError: 'files'

As you saw when I grabbed the data just from my username, it contains a lot of identifying information. I have anonymised and saved this data for a later notebook of analysis

In [64]:
df_min

,author,extensions,message,ohw20_repo
datetime,,,,
2020-08-20 15:08:09+00:00,participant-21,md,Merge pull request #1 from cbirdferrer/patch-1...,False
2020-08-17 16:24:17+00:00,participant-21,ipynb,deleting old presentation_figs file,True
2020-08-14 20:00:38+00:00,participant-21,"ipynb, ipynb",adding improvements to interpolate notebook to...,True
2020-08-14 20:00:38+00:00,participant-21,ipynb,merging changes,True
2020-08-14 20:00:38+00:00,participant-21,"ipynb, ipynb, ipynb, ipynb, ipynb, ipynb, ipyn...",merging interpolate notebook,True
...,...,...,...,...
2020-08-07 15:58:21+00:00,participant-33,md,change read.md,False
2020-08-06 22:26:24+00:00,participant-33,"md, md",change new2.md and create new.md,False
2020-08-06 22:25:02+00:00,participant-33,"md, md",changes in new.md and new2.md,False


-----------------------------

# Going deeper

We can get all the details of a commit by delving deeper into the json structure accessed through the commit url.

**example:**

In [49]:
url = event['payload']['commits'][0]['url']
commit_detail = requests.get(url,auth = (username,token)).json()
commit_detail

{'sha': '9786f1f7fd493e2951ecf8df11f31ac8430d0e7f',
 'node_id': 'MDY6Q29tbWl0MjcwMzg1NzgwOjk3ODZmMWY3ZmQ0OTNlMjk1MWVjZjhkZjExZjMxYWM4NDMwZDBlN2Y=',
 'commit': {'author': {'name': 'Callum Rollo',
   'email': 'c.rollo@outlook.com',
   'date': '2020-06-07T18:06:22Z'},
  'committer': {'name': 'Callum Rollo',
   'email': 'c.rollo@outlook.com',
   'date': '2020-06-07T18:06:22Z'},
  'message': 'Generate Pelican site',
  'tree': {'sha': '8dae3db710ba56571968e5362275537d7eb8d12b',
   'url': 'https://api.github.com/repos/callumrollo/callumrollo.github.io/git/trees/8dae3db710ba56571968e5362275537d7eb8d12b'},
  'url': 'https://api.github.com/repos/callumrollo/callumrollo.github.io/git/commits/9786f1f7fd493e2951ecf8df11f31ac8430d0e7f',
  'comment_count': 0,
  'verification': {'verified': False,
   'reason': 'unsigned',
   'signature': None,
   'payload': None}},
 'url': 'https://api.github.com/repos/callumrollo/callumrollo.github.io/commits/9786f1f7fd493e2951ecf8df11f31ac8430d0e7f',
 'html_url': 'h

The most interesting section is `files`. This gives a summary of the lines changed on each file altered in this commit

In [33]:
commit_detail['files']

[{'sha': '260cdc3ada7e1804883fd4dbc8ff833664bf8884',
  'filename': '404.html',
  'status': 'modified',
  'additions': 5,
  'deletions': 5,
  'changes': 10,
  'blob_url': 'https://github.com/callumrollo/callumrollo.github.io/blob/9786f1f7fd493e2951ecf8df11f31ac8430d0e7f/404.html',
  'raw_url': 'https://github.com/callumrollo/callumrollo.github.io/raw/9786f1f7fd493e2951ecf8df11f31ac8430d0e7f/404.html',
  'contents_url': 'https://api.github.com/repos/callumrollo/callumrollo.github.io/contents/404.html?ref=9786f1f7fd493e2951ecf8df11f31ac8430d0e7f',
  'patch': '@@ -25,8 +25,8 @@\n     <link href="/feeds/all.atom.xml" type="application/atom+xml" rel="alternate" title="Callum Rollo Atom">\n \n \n-    <link rel="shortcut icon" href="https://callumrollo.github.io//images/favicon.ico" type="image/x-icon">\n-    <link rel="icon" href="https://callumrollo.github.io//images/favicon.ico" type="image/x-icon">\n+    <link rel="shortcut icon" href="/images/favicon.ico" type="image/x-icon">\n+    <link 

### Let's go through our little database and extract the file extenstions of all the files altered in each commit

This was originally [Filipe](https://github.com/ocefpaf)'s idea

This requires a small extention to our scraping function. Because we want details at a commit level, this will make an additional API call for every single commit, so adds quite a time penalty. A good target for optimistion with async methods perhaps? *This is left as an excercise for the reader*

In [15]:
def gh_scrape(tgt_users, cred_file = 'credentials.json', verbose=True):
    """Simple scraping function
    Supply a list of github usersnames ['jane-doe', 'torvalds', 'satoshi_nakamoto']
    Returns a dataframe of unique commits by these users over the last 90 days
    Assumes that the github user is the user with most git commits associated with their github profile
    Rate limited by the Github API to 300 events
    Requires you to supply a Github API token in a credentials.json file
    Verbose switch prints a line for each user with the number of events and commits found
    Returns a pandas dataframe of commit info for all usernames in supplied list
    """
    # Get user supplied credentials for the github API
    credentials = json.loads(open(cred_file).read())
    username = credentials['username']
    token = credentials['token']
    df = pd.DataFrame()
    
    for tgt_user in tgt_users:
        base_url = 'https://api.github.com/users/' + tgt_user +'/events'
        url = base_url
        url_list = [base_url]
        data = []
        page_no = 1
        repos_data = []
        total_fetched = 0
        while (True):
            response = requests.get(url,auth = (username,token)).json()
            if type(response)==dict:
                # Catch when the API returns a dict rather than expected list. Usually a credentials error message
                print(response)
                return
            data = data + response
            events_fetched = len(response)
            total_fetched = events_fetched + total_fetched
            if total_fetched == 300:
                # Requesting more will max out the API
                break
            if (events_fetched == 30):
                # if we fethced 30 events from this page, there will be another one after it
                page_no = page_no + 1
                url = base_url + '?page=' + str(page_no)
                url_list.append(url)
            else:
                # We have collected all events by this user
                break
        commits_events = []
        for event in data:
            # We're only interested in commits, which are classed as "PushEvents"
            if event["type"] == "PushEvent":
                commits_events.append(event)
        if len(commits_events)==0:
            # If the user has no commit events, stop processing
            continue
            
        commit_usernames_list = []
        for event in commits_events:
            # Search though the payload for which git user is associated with each commit
            commit_list = event["payload"]["commits"]
            if len(commit_list)>0:    
                for com_n in range(len(commit_list)):
                        commit_username = commit_list[com_n]["author"]["name"]       
                        commit_usernames_list.append(commit_username)
        # Working on the assumption that the git user with the most commits pushed to Github by this user is the one we want
        c = Counter(commit_usernames_list)
        most_common_username = c.most_common(1)[0][0]
        user_commits = []
        
        # Go back through the commits and pull only the ones by the most common git username
        for event in commits_events:
            commit_list = event["payload"]["commits"]
            commit_list_author = []
            if len(commit_list)>0:    
                for com_n in range(len(commit_list)):
                    commit_username = commit_list[com_n]["author"]["name"]
                    if commit_username == most_common_username:
                        commit_list_author.append(commit_list[com_n])
                if commit_list_author:
                    event["payload"]["commits"] = commit_list_author
                    user_commits.append(event)
        
        # Extract the information we're interested in and put it in a pandas DataFrame
        for commit in user_commits:
            for com_n in range(len(commit["payload"]["commits"])):
                commit_detail = commit["payload"]["commits"][com_n]
                commit_all_details = requests.get(commit_detail["url"],auth = (username,token)).json()
                extensions = ""
                for file in commit_all_details['files']:
                    extensions = extensions + file['filename'].split('.')[-1] + ', '
                commit_subset = {"id": commit["id"],
                             "datetime" : commit["created_at"],
                             "sha" : commit_detail["sha"],
                             "message" : commit_detail["message"],
                             "author" : commit_detail["author"]["name"],
                             "url": commit_detail["url"],
                             "repo": commit["repo"]["name"],
                             "extensions": extensions[:-2]}
                df = df.append(commit_subset, ignore_index=True)
        df.index = pd.DatetimeIndex(df.datetime)
        df = df.drop_duplicates(subset=['sha'])
        if verbose:
            print(f"{tgt_user}: found {len(data)} events containing {len(user_commits)} unique commits by {most_common_username}\n")
    return df

In [58]:
users = ["callumrollo"]
df = gh_scrape(users, cred_file = 'credentials-secret.json')
df

callumrollo: found 300 events containing 214 unique commits by Callum Rollo



,author,datetime,extensions,id,message,repo,sha,url
datetime,,,,,,,,
2020-09-06 16:03:54+00:00,Callum Rollo,2020-09-06T16:03:54Z,md,13426925231,Added binder link and to-do list,callumrollo/github-scraper,ed9da46ca18bfe46c77d70647aa08d6a3827db3b,https://api.github.com/repos/callumrollo/githu...
2020-09-05 19:16:44+00:00,Callum Rollo,2020-09-05T19:16:44Z,"ipynb, csv",13422927661,Finalised scraper function,callumrollo/github-scraper,78d24e1e104f616f8026a73a10fd525e67019342,https://api.github.com/repos/callumrollo/githu...
2020-09-05 19:16:44+00:00,Callum Rollo,2020-09-05T19:16:44Z,"ipynb, ipynb",13422927661,detailed time breakdown histograms,callumrollo/github-scraper,2445a3d0d800bc3fea933d713ae5cca173bd0f72,https://api.github.com/repos/callumrollo/githu...
2020-09-05 17:21:13+00:00,Callum Rollo,2020-09-05T17:21:13Z,"html, html, html, html, xml, xml, xml, xml, ht...",13422558148,Generate Pelican site,callumrollo/callumrollo.github.io,a9e3bb9213139f467fb5058847ff063e851b1ce6,https://api.github.com/repos/callumrollo/callu...
2020-09-05 17:20:55+00:00,Callum Rollo,2020-09-05T17:20:55Z,"md, md",13422557015,updated categories,callumrollo/callumrollo.github.io,aca1fa72ad196df5351661a971b7de8add348fc6,https://api.github.com/repos/callumrollo/callu...
...,...,...,...,...,...,...,...,...
2020-06-08 12:32:52+00:00,Callum Rollo,2020-06-08T12:32:52Z,"png, aux, log, out, pdf, gz, tex, pdf, html, j...",12567171570,Deployed 63f61e7 with MkDocs version: 1.1.2,ueapy/pythoncourse2020,21602fb12ce2483e8e6faf3af926345a6359d754,https://api.github.com/repos/ueapy/pythoncours...
2020-06-08 12:32:28+00:00,Callum Rollo,2020-06-08T12:32:28Z,"png, pdf, tex, md",12567167762,ENV East >> Aries,ueapy/pythoncourse2020,63f61e7f654217d5cdc1af9925de2ed593fe154e,https://api.github.com/repos/ueapy/pythoncours...
2020-06-08 10:52:40+00:00,Callum Rollo,2020-06-08T10:52:40Z,md,12566251693,typo,callumrollo/callumrollo.github.io,e712699f1d08f2294bd7e462bcb92768f6841e04,https://api.github.com/repos/callumrollo/callu...


Success! We have the filetypes used in every commit by this author. This should tell us, in broad strokes, what programming language they are working on and potentially many other things. Are they working on .md files for documentation? Uploading lots of .png iamges? Do they prefer `.ipynb` notebooks to pure `.py` files?

### Ideas for further analysis
- Use of different filetypes. Particularly .py vs .ipynb
- word cloud of commit messages
- check out non-commit activity: merge, PR, issue...
- geographical/timezone patterns
- how much "crunch" did we get before presentations on Friday?
- examine links between authors (who merged who? Comments mentioning issues?)

### Github is a rich mine of information
I hope this notebook has given you some ideas of the kinds of information you can grab from GitHub. It may also serve as a good reminder that everything we put on there is public and scrapable, so write helpful commit messages!

In [3]:
HTML(html)